# TP2 NMT - CALVEZ Brieuc, ARDIET Simon

In [ ]:
!pip install OpenNMT-py

     |████████████████████████████████| 216 kB 5.4 MB/s 
     |████████████████████████████████| 73 kB 1.7 MB/s 
     |████████████████████████████████| 56 kB 563 kB/s 
     |████████████████████████████████| 16.3 MB 47.3 MB/s 
     |████████████████████████████████| 1.2 MB 24.8 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Ensimag/'3A Ensimag'/'Traitement du langage'/2-tp_nmt

/content/drive/MyDrive/Ensimag/3A Ensimag/Traitement du langage/2-tp_nmt


In [ ]:
%cd drive/MyDrive/Cours/3A/Traitement-langage/TP2

/content/drive/MyDrive/Cours/3A/Traitement-langage/TP2


In [ ]:
%ls

BTEC-en-fr/            pred1.txt          pred64Unites.txt
config-base.yaml       pred2Layers.txt    predBeamSize.txt
models/                pred2.txt          predDev.txt
multi-bleu.perl        pred3Layers.txt    predEarlyStopping.txt
nonbreaking_prefix.en  pred3.txt          README.md
nonbreaking_prefix.fr  pred4.txt          tokenizer.perl
pred128Unites.txt      pred512Unites.txt


## Question 1

Ce sont des fichiers qui contiennent des données pour le *training* du réseau. Le premier contient des phrases en français ; l'autre les mêmes phrases en anglais. Toutes les phrases sont triées et identifiées par un id unique de la forme `TRAIN_XXXXX`

## Question 2

In [ ]:
# %ls BTEC-en-fr

In [ ]:
%%shell
awk -F '\' '{print $NF}' BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.txt > BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.clean.txt
awk -F '\' '{print $NF}' BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.txt > BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.clean.txt
perl tokenizer.perl -l en -lc < ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.clean.txt > ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt
perl tokenizer.perl -l fr -lc < ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.clean.txt > ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt
awk -F '\' '{print $NF}' BTEC-en-fr/test/IWSLT09_BTEC.testset.en.txt > BTEC-en-fr/test/IWSLT09_BTEC.testset.en.clean.txt
awk -F '\' '{print $NF}' BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.txt > BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.clean.txt
perl tokenizer.perl -l en -lc < ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.clean.txt > ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt
perl tokenizer.perl -l fr -lc < ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.clean.txt > ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt
awk -F '\' '{print $NF}' BTEC-en-fr/train/IWSLT10_BTEC.train.en.txt > BTEC-en-fr/train/IWSLT10_BTEC.train.en.clean.txt
awk -F '\' '{print $NF}' BTEC-en-fr/train/IWSLT10_BTEC.train.fr.txt > BTEC-en-fr/train/IWSLT10_BTEC.train.fr.clean.txt
perl tokenizer.perl -l en -lc < ./BTEC-en-fr/train/IWSLT10_BTEC.train.en.clean.txt > ./BTEC-en-fr/train/IWSLT10_BTEC.train.en.tok.txt
perl tokenizer.perl -l fr -lc < ./BTEC-en-fr/train/IWSLT10_BTEC.train.fr.clean.txt > ./BTEC-en-fr/train/IWSLT10_BTEC.train.fr.tok.txt

Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: fr
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: fr
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: fr
Number of threads: 1


Pour chaque fichier :

*   la première commande sert à enlever les identifiants uniques qui se trouvent au début de chaque ligne des fichiers ;
*   la seconde commande sert à espacer les ponctuations de sorte à ce qu'elles ne touchent plus les mots, et faire en sorte que quelques caractères soient *tokensisés* (par exemple, `'` deviant `&apos;`, etc.)

L'intérêt de cette opération est de bien préparer les modèles avant de le traiter dans le réseau de neurones. Ainsi, la ponctuation ne sera pas pris en compte alors que les apostrophes le seront (puisqu'elles font partie intégrantes des traductions).

<span style="color:red"> Le but de cette stratégie basique de tokenization est de réduire la taille du vocabulaire. Si on ne sépare pas les mots et la ponctuation on sera obligé à avoir plus de tokens dans le vocabularie. E.g. "Salut", "Salut!", "Salut,", "Salut?", etc. </span>



## Question 3

Les jeux de données d'entraînement, de validation et de test doivent être différents afin d'éviter **l'overfitting**:
- Le jeu d'entraînement est utilisé durant le processus d'apprentissage et permet d'établir les paramètres (poids, biais..).
- Le jeu de validation permet lui d'établir les hyperparamètres. Aussi, si le jeu d'entraînement renvoie plusieurs classificateurs, le jeu de validation permettra de les comparer et de choisir le plus adéquat.
- Le jeu de test permet lui de mesurer les performances du classificateur entièrement spécifié.

## Question 4

In [ ]:
%%shell
onmt_build_vocab -config config-base.yaml -n_sample -1

Corpus train's weight should be given. We default it to 1 for you.
[2022-02-15 09:43:05,510 INFO] Counter vocab from -1 samples.
[2022-02-15 09:43:05,510 INFO] n_sample=-1: Build vocab on full datasets.
[2022-02-15 09:43:05,522 INFO] train's transforms: TransformPipe()
[2022-02-15 09:43:05,828 INFO] Counters src:9978
[2022-02-15 09:43:05,828 INFO] Counters tgt:8194


Le système de traduction automatique a besoin d'un vocabulaire afin de spécifier les termes et mots du langage. On pourra alors effectuer des distributions, les comparer, en déduire des vecteurs etc. Il est préférable de se baser sur le données d'apprentissage, puisque dans un monde idéal, le vocabulaire ne comprendrait que les mots de l'ensemble d'apprentissage. Tous les mots inconnus se verraient attribuer un token `unknown`. On gagne alors en efficacité.

<span style="color:red">Ca veut dire quoi "dans un monde idéal, le vocabulaire ne comprendrait que les mots de l'ensemble d'apprentissage"? Les tokens du vocabulaire basé sur l'ensemble d'apprentissage sont suffisants pour l’entraînement du modèle parce que pendent l’entraînement le modèle ne voit que des phrases issues du training set. En autre, utiliser un vocabulaire plus vaste serait inefficace et gourmand en calcul et en mémoire, car des tokens de ce vocabulaire ne seront jamais utilisés pour l’entraînement.</span> 

## Question 5

*   `Counter src` nous donne le nombre de token dans la source
*   `Counter tgt` nous donne le nombre de token dans la target

<span style="color:red">plus précisement: le nombre de tokens uniques présents dans le vocabulaire source et cible.</span> 

In [ ]:
%%shell
onmt_train -config config-base.yaml

[2022-02-21 09:01:57,075 INFO] Missing transforms field for train data, set to default: [].
[2022-02-21 09:01:57,233 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-21 09:01:57,233 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-21 09:01:57,234 INFO] Parsed 2 corpora from -data.
[2022-02-21 09:01:57,235 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-21 09:01:57,235 INFO] Loading vocab from text file...
[2022-02-21 09:01:57,235 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-21 09:01:57,527 INFO] Loaded src vocab has 9978 tokens.
[2022-02-21 09:01:57,535 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-21 09:01:57,779 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-21 09:01:57,784 INFO] Building fields with vocab in counters...
[2022-02-21 09:01:57,794 INFO]  * tgt vocab size: 8198.
[2022-02-21 09:01:57,814 INFO]  * src vocab size: 

## Question 6

La valeur `acc` correspond à l'accuracy, donc la précision du modèle. Dans `statistics.py`, elle est calculée comme ceci : 
```python
return 100 * (self.n_correct / self.n_words)
```

On souhaite donc que sa valeur soit élevée. 

La valeur `ppl` correspond à la `perplexity`. Dans `statistics.py`, elle est calculée comme ceci : 
```python
math.exp(min(self.loss / self.n_words, 100))
```

On souhaite donc que sa valeur soit faible, pour minimiser les pertes

<span style="color:red">Intuition dérrière la définition de perplexity?</span> 

## Question 7

*   `train_step` : Nombre de pas d'entraînement (par défaut : 100000). <span style="color:red">C'est quoi un pas d'entraînement? (vous n'avez que traduit le nom de la variable sans expliquer sa signification). train_step = nombre total d'étapes d'optimisation des poids du modèle, où chaque étape est constituée d'un passage en avant d'un lot (batch) de données d'apprentissage dans le modèle (forward pass) et d'un passage en arrière des gradients générés par la fonction de perte (backward pass), suivi par une mise à jours des poids du modèle. </span>
*   `valid_step` : Nombre de pas à partir duquel la validation est effectuée (par défaut : 10000). <span style="color:red">nombre d'étapes d'entrainement entre une étape de validation et la suivante. </span>
*   `enc_layers` : Nombre de couches dans l'encodeur (par défaut : 2).
*   `dec_layers` : Nombre de couches dans le décodeur (par défaut : 2).
*   `enc_rnn_size` : Taille des états cachés de l'encodeur RNN (par défaut : 500).
*   `dec_rnn_size` : Taille des états cachés du décodeur RNN (par défaut : 500).
*   `batch_size` : Taille maximum du batch pour l'entraînement (par défaut : 64). La batch size définit le nombre d'échantillons qui seront propagés dans le réseau. On peut utiliser une batch size inférieure au nombre d'échantillon, pour utiliser moins de mémoire. Cependant, l'inconvénient principal est qu'une batch size trop petite entraînera une moins bonne estimation du gradient.

In [ ]:
%%shell
# ls models/base
onmt_translate -model models/base/model_step_625.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred1.txt
onmt_translate -model models/base/model_step_1250.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred2.txt
onmt_translate -model models/base/model_step_1875.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred3.txt
onmt_translate -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred4.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-21 09:08:19,358 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-21 09:08:24,199 INFO] PRED AVG SCORE: -1.3401, PRED PPL: 3.8193
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout opti

## Question 8

Le score BLEU est compris entre 0 et 1 et indique la similitude du texte « candidat » par rapport aux textes de « référence ». 
Plus le score se rapproche de 1, plus les textes sont similaires. Le score BLEU semble donc adapté à notre cas.

NB: Dans la suite du TP, les scores BLEU seront exprimés en pourcentage (entre 0 et 100%).

## Question 9

Calculons les scores BLEU (exprimés en pourcentage)

In [ ]:
%%shell
# ls BTEC-en-fr/test
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < pred1.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < pred2.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < pred3.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < pred4.txt

BLEU = 5.12, 25.2/7.5/3.6/1.0 (BP=1.000, ratio=1.508, hyp_len=5391, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 13.08, 45.5/18.6/10.4/3.9 (BP=0.962, ratio=0.963, hyp_len=3440, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 18.56, 51.9/23.8/15.2/8.2 (BP=0.937, ratio=0.938, hyp_

C'est le troisième modèle qui a le meilleur score (18.56), suivi de près par le quatrième (18.26)

## Question 10

Dans une certaine mesure, le score BLEU augmente avec le nombre de pas. En effet, il est souvent maximisé pour quelques dizaines de milliers de pas. Cependant, atteindre des centaines de milliers de pas est inutile, d'où l'intérêt du "early stopping" qui optimise le temps de calcul.

Dans le fichier `config-base.yaml`, on ajoute les options suivantes :

```
early_stopping: 5
early_stopping_criteria: [accuracy, ppl]
```

La première ligne veut dire que l'on souhaite ne constater aucune amélioration durant 5 steps pour arrêter l'entrainement. Les améliorations sont basées sur la seconde ligne.

In [ ]:
%%shell
onmt_build_vocab -config config-base.yaml -n_sample -1
onmt_train -config config-base.yaml

Corpus train's weight should be given. We default it to 1 for you.
[2022-02-18 16:10:14,513 INFO] Counter vocab from -1 samples.
[2022-02-18 16:10:14,513 INFO] n_sample=-1: Build vocab on full datasets.
[2022-02-18 16:10:14,526 INFO] train's transforms: TransformPipe()
[2022-02-18 16:10:14,834 INFO] Counters src:9978
[2022-02-18 16:10:14,834 INFO] Counters tgt:8194
[2022-02-18 16:10:15,146 WARNING] path ./BTEC-en-fr/BTEC-en-fr/vocab.src exists, may overwrite...
[2022-02-18 16:10:15,707 WARNING] path ./BTEC-en-fr/BTEC-en-fr/vocab.tgt exists, may overwrite...
[2022-02-18 16:10:17,975 INFO] Missing transforms field for train data, set to default: [].
[2022-02-18 16:10:18,421 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-18 16:10:18,421 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-18 16:10:18,422 INFO] Parsed 2 corpora from -data.
[2022-02-18 16:10:18,423 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': se

Dans le fichier de log, on constate bien que régulièrement, un `Decreasing patience : */5` apparait régulièrement. On avait augmenté le nombre de trainig steps à 20000 dans le `config-base.yaml`, et avec notre critère d'arrêt anticipé, c'est à l'étape 13740/20000 que l'entrainement s'arrête.

Dans la suite, on va donc se baser sur un nombre d'étapes de 13740.

Le fichier de log indique également que le meilleur modèle a été trouvé à l'étape 9375. Si l'on exécute les commandes précédentes, on constate la chose suivante :

In [ ]:
%%shell
onmt_translate -model models/base/model_step_9375.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output predEarlyStopping.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-18 16:20:34,453 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-18 16:20:39,207 INFO] PRED AVG SCORE: -0.4891, PRED PPL: 1.6309


In [ ]:
%%shell
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < predEarlyStopping.txt

BLEU = 34.87, 64.9/41.0/29.9/20.7 (BP=0.973, ratio=0.973, hyp_len=3478, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


Soit un score BLEU de 34.87%. C'est en nette amélioration !

## Question 11

Dans le `config-base.yaml`, on change les lignes pour l'encodeur et le décodeur de cette façon :
```
enc_layers: 2
dec_layers: 2
```
Les tailles de ces couchent passent maintenant à 2. De plus, on commente les lignes qui étaient utilisées pour le *early stopping* à la question précédente. Entrainons maintenant notre réseau.

In [ ]:
%%shell
onmt_build_vocab -config config-base.yaml -n_sample -1
onmt_train -config config-base.yaml

Corpus train's weight should be given. We default it to 1 for you.
[2022-02-18 16:25:01,992 INFO] Counter vocab from -1 samples.
[2022-02-18 16:25:01,992 INFO] n_sample=-1: Build vocab on full datasets.
[2022-02-18 16:25:02,005 INFO] train's transforms: TransformPipe()
[2022-02-18 16:25:02,312 INFO] Counters src:9978
[2022-02-18 16:25:02,312 INFO] Counters tgt:8194
[2022-02-18 16:25:02,313 WARNING] path ./BTEC-en-fr/BTEC-en-fr/vocab.src exists, may overwrite...
[2022-02-18 16:25:02,331 WARNING] path ./BTEC-en-fr/BTEC-en-fr/vocab.tgt exists, may overwrite...
[2022-02-18 16:25:04,135 INFO] Missing transforms field for train data, set to default: [].
[2022-02-18 16:25:04,137 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-18 16:25:04,137 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-18 16:25:04,138 INFO] Parsed 2 corpora from -data.
[2022-02-18 16:25:04,138 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': se

In [ ]:
%%shell
onmt_translate -model models/base/model_step_9375.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred2Layers.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < pred2Layers.txt

[2022-02-18 16:35:23,165 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-18 16:35:28,340 INFO] PRED AVG SCORE: -0.5588, PRED PPL: 1.7486
BLEU = 31.46, 62.2/37.9/27.3/18.1 (BP=0.957, ratio=0.958, hyp_len=3425, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores fr

En passant le nombre de couches (encodage et décodage) à 2, on constate une baisse du score BLEU par rapport à avant. On passe maintenant à 31,46%. Essayons voir avec trois couches.

In [ ]:
%%shell
onmt_build_vocab -config config-base.yaml -n_sample -1
onmt_train -config config-base.yaml
onmt_translate -model models/base/model_step_9375.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred3Layers.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < pred3Layers.txt

Corpus train's weight should be given. We default it to 1 for you.
[2022-02-18 16:37:53,639 INFO] Counter vocab from -1 samples.
[2022-02-18 16:37:53,639 INFO] n_sample=-1: Build vocab on full datasets.
[2022-02-18 16:37:53,652 INFO] train's transforms: TransformPipe()
[2022-02-18 16:37:54,058 INFO] Counters src:9978
[2022-02-18 16:37:54,058 INFO] Counters tgt:8194
[2022-02-18 16:37:54,058 WARNING] path ./BTEC-en-fr/BTEC-en-fr/vocab.src exists, may overwrite...
[2022-02-18 16:37:54,077 WARNING] path ./BTEC-en-fr/BTEC-en-fr/vocab.tgt exists, may overwrite...
[2022-02-18 16:37:55,946 INFO] Missing transforms field for train data, set to default: [].
[2022-02-18 16:37:55,948 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-18 16:37:55,949 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-18 16:37:55,949 INFO] Parsed 2 corpora from -data.
[2022-02-18 16:37:55,950 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': se

Cette fois, on a un score BLEU de 27,39%

On constate donc que plus il y a de couches, moins bons sont les résultats. On suppose que c'est dû au fait que quand on ajoute des couches, rajoute des calculs et de l'information à traiter, qui ici ne semble pas utile à notre modèle, et à l'inverse gêne à son ben entrainement. Le modèle restant relativement simple, une unique couche d'encodage-décodage offre le meilleur rapport (temps de calcul/qualité de traduction).

<span style="color:red">Il aurait fallu utiliser l'option early_stopping pour bien repondre à cette question (ainsi que la prochaine). En effet, si l'on utilise pas cette option, on risque de comparer des modèles qui sont sous-entrainé ou sur-entrainés (overfitted),  sans donc pouvoir en tirer des conclusions. En effet, le nombre d'étapes d'entrainement nécessaires à l'optimization du modèle varie selon la quantité de paramétres (poids) à optimizer, ce qui est le cas lorsqu'on varie le nombre de couches ou leur taille.  </span>


## Question 12

Dans cette question, on rebascule le nombre de couches du decoder et de l'encoder à 1 (comme cela était fait initialement). On modifie également le `config-base.yaml` de la façon suivante :
```
rnn_type: LSTM
dec_rnn_size: 256
enc_rnn_size: 256
```
Le nombre `256` est là initialement, on va changer en `128` dans un premier temps pour voir l'évolution du score BLEU.

In [ ]:
%%shell
onmt_build_vocab -config config-base.yaml -n_sample -1
onmt_train -config config-base.yaml
onmt_translate -model models/base/model_step_9375.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred128Unites.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < pred128Unites.txt

Corpus train's weight should be given. We default it to 1 for you.
[2022-02-19 08:49:40,976 INFO] Counter vocab from -1 samples.
[2022-02-19 08:49:40,976 INFO] n_sample=-1: Build vocab on full datasets.
[2022-02-19 08:49:40,987 INFO] train's transforms: TransformPipe()
[2022-02-19 08:49:41,993 INFO] Counters src:9978
[2022-02-19 08:49:41,993 INFO] Counters tgt:8194
[2022-02-19 08:49:42,243 WARNING] path ./BTEC-en-fr/BTEC-en-fr/vocab.src exists, may overwrite...
[2022-02-19 08:49:42,572 WARNING] path ./BTEC-en-fr/BTEC-en-fr/vocab.tgt exists, may overwrite...
[2022-02-19 08:49:45,397 INFO] Missing transforms field for train data, set to default: [].
[2022-02-19 08:49:45,790 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-19 08:49:45,791 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-19 08:49:45,792 INFO] Parsed 2 corpora from -data.
[2022-02-19 08:49:45,792 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': se

On a un score BLEU de 30,15 ! C'est en baisse par rapport à ce qu'on avait obtenu à la question 10 (34,87%)

On se propose alors de passer le nombre d'unités du decodeur et de l'encodeur à XXX (modification toujours faite dans le `config-base.yaml`) pour voir si la tendance se confirme, à savoir que moins il y a d'instances, moins le score BLEU est élevé.

In [ ]:
%%shell
onmt_build_vocab -config config-base.yaml -n_sample -1
onmt_train -config config-base.yaml
onmt_translate -model models/base/model_step_9375.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred64Unites.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < pred64Unites.txt

Corpus train's weight should be given. We default it to 1 for you.
[2022-02-19 08:56:11,333 INFO] Counter vocab from -1 samples.
[2022-02-19 08:56:11,333 INFO] n_sample=-1: Build vocab on full datasets.
[2022-02-19 08:56:11,344 INFO] train's transforms: TransformPipe()
[2022-02-19 08:56:11,657 INFO] Counters src:9978
[2022-02-19 08:56:11,657 INFO] Counters tgt:8194
[2022-02-19 08:56:11,658 WARNING] path ./BTEC-en-fr/BTEC-en-fr/vocab.src exists, may overwrite...
[2022-02-19 08:56:11,673 WARNING] path ./BTEC-en-fr/BTEC-en-fr/vocab.tgt exists, may overwrite...
[2022-02-19 08:56:13,202 INFO] Missing transforms field for train data, set to default: [].
[2022-02-19 08:56:13,203 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-19 08:56:13,204 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-19 08:56:13,204 INFO] Parsed 2 corpora from -data.
[2022-02-19 08:56:13,205 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': se

Le score BLEU est de 21,31% : nos attentes sont confirmées. Passons maintenant le nombre d'instances à 512 pour voir si on observerait la tendance inverse.

In [ ]:
%%shell
onmt_build_vocab -config config-base.yaml -n_sample -1
onmt_train -config config-base.yaml
onmt_translate -model models/base/model_step_9375.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred512Unites.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < pred512Unites.txt

Corpus train's weight should be given. We default it to 1 for you.
[2022-02-19 09:01:15,298 INFO] Counter vocab from -1 samples.
[2022-02-19 09:01:15,299 INFO] n_sample=-1: Build vocab on full datasets.
[2022-02-19 09:01:15,309 INFO] train's transforms: TransformPipe()
[2022-02-19 09:01:15,615 INFO] Counters src:9978
[2022-02-19 09:01:15,615 INFO] Counters tgt:8194
[2022-02-19 09:01:15,615 WARNING] path ./BTEC-en-fr/BTEC-en-fr/vocab.src exists, may overwrite...
[2022-02-19 09:01:15,631 WARNING] path ./BTEC-en-fr/BTEC-en-fr/vocab.tgt exists, may overwrite...
[2022-02-19 09:01:17,292 INFO] Missing transforms field for train data, set to default: [].
[2022-02-19 09:01:17,294 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-19 09:01:17,294 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-19 09:01:17,295 INFO] Parsed 2 corpora from -data.
[2022-02-19 09:01:17,295 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': se

Le score BLEU est de 33,46%, c'est toujours moins bon qu'avant, mais le résultat est moins catastrophique qu'en diminuant le nombre d'instances.

Nous supposons que, comme pour le beam search que nous allons voir dans la suite, les hyperparamètres modulables ne sont pas "binaires". Il faut les manipuler subtilement pour trouver une valeur optimale, et non pas les faire croître ou décroîte au maximum. 



## Question 13

**Cette question n'a pas été traitée.**

## Question 14

Le beam search (i.e recherche en faisceaux) permet d'explorer un graphe en limitant le nombres de fils pour chaque noeud. Il s'agit d'un algorithme d'optimisation d'exploration heuristique, qui réduit drastiquement la mémoire utilisée lors de la recherche d'une solution. 

<span style="color:red"> C'est quoi les noeuds et les fils dans notre cas ? --> les tokens de chaque étape de décodage de la phrase cible, c.à.d. de la traduction </span>


Sa taille par défaut, selon la documentation de OMT, est de 5.

## Question 15



Repassons les paramètres du `config-base.yaml` par défaut (comme c'était à la fin de la question 10) et regénérons le `model_step_9375.pt` (celui qui nous donnait les meilleurs résultats).



In [ ]:
%%shell
onmt_build_vocab -config config-base.yaml -n_sample -1
onmt_train -config config-base.yaml

Corpus train's weight should be given. We default it to 1 for you.
[2022-02-19 09:21:27,916 INFO] Counter vocab from -1 samples.
[2022-02-19 09:21:27,916 INFO] n_sample=-1: Build vocab on full datasets.
[2022-02-19 09:21:27,927 INFO] train's transforms: TransformPipe()
[2022-02-19 09:21:28,232 INFO] Counters src:9978
[2022-02-19 09:21:28,233 INFO] Counters tgt:8194
[2022-02-19 09:21:28,233 WARNING] path ./BTEC-en-fr/BTEC-en-fr/vocab.src exists, may overwrite...
[2022-02-19 09:21:28,247 WARNING] path ./BTEC-en-fr/BTEC-en-fr/vocab.tgt exists, may overwrite...
[2022-02-19 09:21:29,766 INFO] Missing transforms field for train data, set to default: [].
[2022-02-19 09:21:29,767 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-19 09:21:29,767 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-19 09:21:29,768 INFO] Parsed 2 corpora from -data.
[2022-02-19 09:21:29,768 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': se

Maintenant, on ajoute l'option `-beam_size X`, où on va faire varier `X` entre 1 et 10.

In [ ]:
%%shell
echo "------- 1 -------"
onmt_translate -beam_size 1 -model models/base/model_step_13740.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output predBeamSize.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < predBeamSize.txt
echo "------- 2 -------"
onmt_translate -beam_size 2 -model models/base/model_step_13740.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output predBeamSize.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < predBeamSize.txt
echo "------- 3 -------"
onmt_translate -beam_size 3 -model models/base/model_step_13740.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output predBeamSize.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < predBeamSize.txt
echo "------- 4 -------"
onmt_translate -beam_size 4 -model models/base/model_step_13740.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output predBeamSize.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < predBeamSize.txt
echo "------- 5 -------"
onmt_translate -beam_size 5 -model models/base/model_step_13740.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output predBeamSize.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < predBeamSize.txt
echo "------- 6 -------"
onmt_translate -beam_size 6 -model models/base/model_step_13740.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output predBeamSize.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < predBeamSize.txt
echo "------- 7 -------"
onmt_translate -beam_size 7 -model models/base/model_step_13740.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output predBeamSize.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < predBeamSize.txt
echo "------- 8 -------"
onmt_translate -beam_size 8 -model models/base/model_step_13740.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output predBeamSize.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < predBeamSize.txt
echo "------- 9 -------"
onmt_translate -beam_size 9 -model models/base/model_step_13740.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output predBeamSize.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < predBeamSize.txt
echo "------- 10 -------"
onmt_translate -beam_size 10 -model models/base/model_step_13740.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output predBeamSize.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < predBeamSize.txt

------- 1 -------
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-19 09:41:13,304 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-19 09:41:14,823 INFO] PRED AVG SCORE: -0.5109, PRED PPL: 1.6668
BLEU = 32.01, 60.0/36.6/26.2/18.3 (BP=1.000, ratio=1.042, hyp_len=3724, re

En faisant varier la beam size de 1 à 10, le score BLEU est croissant puis décroissant, avec un maximum en 8 pour un score BLEU de 34,99%.

<span style="color:red"> Pourquoi? </span>


## Question 16

La combinaison la plus efficace est sauvegardée dans le `model_step_13740.pt`. Il suffit alors de reprendre la question précédente et d'utiliser `translate` avec un *beam size* de 8.

In [ ]:
%%shell
onmt_translate -beam_size 8 -model models/base/model_step_13740.pt -src BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output predDev.txt
perl multi-bleu.perl BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < predDev.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-21 08:34:12,260 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-21 08:34:19,579 INFO] PRED AVG SCORE: -0.4281, PRED PPL: 1.5343
BLEU = 35.04, 67.6/43.0/31.0/22.0 (BP=0.934, ratio=0.936, hyp_len=3563, ref_len=3808)
It is 

Le meilleur score BLEU que l'on peut obtenir sur dev est donc de 35,04%.

Calculons le score BLEU sur l'ensemble de test :

In [ ]:
%%shell
onmt_translate -beam_size 8 -model models/base/model_step_13740.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output predDev.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < predDev.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-21 08:51:54,642 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-21 08:52:01,464 INFO] PRED AVG SCORE: -0.4228, PRED PPL: 1.5263
BLEU = 34.99, 65.2/41.6/31.0/22.4 (BP=0.945, ratio=0.946, hyp_len=3381, ref_len=3574)
It is 

On retrouve bien notre score BLEU de 34,99%. On aurait aimé en avoir un meilleur, mais au vu de l'échantillon, on se doute bien qu'il sera compliqué de se rapprocher des 100%.

## Question 17

On repasse le `config-base.yaml` dans sa configuration initiale et on recalcule le tout. Puis on traduit sur le même modèle qu'à la question précédente pour comparer.

In [ ]:
%%shell
onmt_train -config config-base.yaml
onmt_translate -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred4.txt

[2022-02-21 09:12:03,572 INFO] Missing transforms field for train data, set to default: [].
[2022-02-21 09:12:03,574 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-21 09:12:03,574 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-21 09:12:03,575 INFO] Parsed 2 corpora from -data.
[2022-02-21 09:12:03,576 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-21 09:12:03,576 INFO] Loading vocab from text file...
[2022-02-21 09:12:03,576 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-21 09:12:03,605 INFO] Loaded src vocab has 9978 tokens.
[2022-02-21 09:12:03,610 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-21 09:12:03,632 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-21 09:12:03,637 INFO] Building fields with vocab in counters...
[2022-02-21 09:12:03,646 INFO]  * tgt vocab size: 8198.
[2022-02-21 09:12:03,660 INFO]  * src vocab size: 

On a donc juste à comparer le `pred4.txt` que l'on vient de regénérer avec le `predDev.txt` de la précédente. Le résultat est sans appel, les phrases n'ont aucun rapport dans les deux modèles !

On a l'impression — justifiée — que notre modèle entrainé à la question précédente est bien meilleur que le modèle initial.

La comparaison des lignes 10 est amusante. Dans le modèle initial (question 6) :

```
Japanese people like to have some medicine .
```

Alors que dans le meilleur :

```
Japanese teenagers are interested in Japanese .
```

On peut prendre un autre exemple pour illustrer notre propos. À la ligne 18, dans le moins bon modèle :

```
I have a reservation .
```

Alors que dans le meilleur :

```
I am nauseous .
```
